In [ ]:
!pip install natasha rdflib

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 76.7 MB/s eta 0:00:00:00:01:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsNERTagger, Doc

df = pd.read_excel("/kaggle/input/rbc-news-f/rbc_news_firts_week.xlsx")
texts = df["text"].tolist()

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

ner_results = []

for text in texts:
    if not isinstance(text, str) or not text.strip():
        ner_results.append({"text": text, "entities": []})
        continue

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.tag_ner(ner_tagger)

    entities = []
    for span in doc.spans:
        if span.type == 'ORG':
            normal_form = span.text
        else:
            tokens = span.tokens
            lemmas = [token.lemma for token in tokens]
            normal_form = ' '.join(lemmas)
        entities.append({
            "text": span.text,
            "type": span.type,
            "normal_form": normal_form
        })
    ner_results.append({"text": text, "entities": entities})

all_entities = []
for result in ner_results:
    for entity in result["entities"]:
        all_entities.append({
            "text": result["text"],
            "entity_text": entity["text"],
            "entity_type": entity["type"],
            "entity_normal_form": entity["normal_form"]
        })

df_ner = pd.DataFrame(all_entities)
df_ner.to_csv("ner_results.csv", index=False)

In [ ]:
a = pd.read_csv('/kaggle/working/ner_results.csv')
a

,text,entity_text,entity_type,entity_normal_form
0,Взрыв газа произошел в одной из квартир пятиэт...,Луначарского,LOC,луначарский
1,Взрыв газа произошел в одной из квартир пятиэт...,Петрозаводске,LOC,петрозаводск
2,Взрыв газа произошел в одной из квартир пятиэт...,РИА Новости,ORG,РИА Новости
3,Взрыв газа произошел в одной из квартир пятиэт...,МЧС,ORG,МЧС
4,Взрыв газа произошел в одной из квартир пятиэт...,РЕН ТВ,ORG,РЕН ТВ
...,...,...,...,...
28790,Патриарх Московский и всея Руси Кирилл в Рожде...,Санкт-Петербурга,LOC,санкт-петербург
28791,Патриарх Московский и всея Руси Кирилл в Рожде...,Рэй Далио,PER,рэй далио
28792,Патриарх Московский и всея Руси Кирилл в Рожде...,США,LOC,сша
28793,Патриарх Московский и всея Руси Кирилл в Рожде...,Трампа,PER,трамп


In [ ]:
import pandas as pd
import requests
import time

def get_wikidata_id(entity_normal_form, entity_type, lang='ru'):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'search': entity_normal_form,
        'language': lang,
        'format': 'json'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return data['search'][0]['id'] if data.get('search') else None
    except Exception as e:
        print(f"Ошибка при запросе для {entity_text}: {e}")
        return None

df_ner = pd.read_csv("ner_results.csv")

nel_data = []

# Process the first 5 entities
for idx, row in df_ner.iloc[:5].iterrows():
    entity_text = row['entity_text']
    entity_type = row['entity_normal_form']
    qid = get_wikidata_id(entity_type, entity_type)
    nel_data.append({
        'original_text': row['text'],
        'entity_text': entity_text,
        'entity_normal_form': entity_type,
        'wikidata_qid': qid
    })
    time.sleep(1)

df_nel = pd.DataFrame(nel_data)
df_nel.to_csv("nel_results.csv", index=False)

In [ ]:
df_nel

,original_text,entity_text,entity_normal_form,wikidata_qid
0,Взрыв газа произошел в одной из квартир пятиэт...,Луначарского,луначарский,Q747751
1,Взрыв газа произошел в одной из квартир пятиэт...,Петрозаводске,петрозаводск,Q1895
2,Взрыв газа произошел в одной из квартир пятиэт...,РИА Новости,РИА Новости,Q821172
3,Взрыв газа произошел в одной из квартир пятиэт...,МЧС,МЧС,Q2624171
4,Взрыв газа произошел в одной из квартир пятиэт...,РЕН ТВ,РЕН ТВ,Q1479649


In [ ]:
dfa = pd.read_excel('/kaggle/input/rbc-news-f/rbc_news_firts_week.xlsx')
df = dfa.head(1)

In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD
from urllib.parse import quote
import hashlib

WD = Namespace("http://www.wikidata.org/entity/")
SCHEMA = Namespace("http://schema.org/")
NST = Namespace("http://example.org/ns/")

def create_rdf_graph(nel_csv_path, output_path="knowledge_graph.ttl"):
    g = Graph()
    g.bind("wd", WD)
    g.bind("schema", SCHEMA)
    g.bind("nst", NST)

    for idx, row in df_nel.iterrows():
        if pd.notna(row['wikidata_qid']):
            entity_uri = URIRef(WD[row['wikidata_qid']])

            class_name = quote(row['entity_normal_form'].strip().replace(' ', '_'), safe='')
            class_uri = NST[class_name]

            g.add((entity_uri, RDF.type, class_uri))

            text_hash = hashlib.md5(row['original_text'].encode()).hexdigest()
            doc_uri = URIRef(f"http://example.org/news/{text_hash}")

            g.add((doc_uri, RDF.type, SCHEMA.NewsArticle))
            g.add((doc_uri, SCHEMA.text, Literal(row['original_text'], lang="ru")))
            g.add((doc_uri, SCHEMA.mentions, entity_uri))

            g.add((entity_uri, SCHEMA.name, Literal(row['entity_text'], lang="ru")))
            g.add((entity_uri, NST.normalizedForm, Literal(row['entity_normal_form'])))

    g.serialize(destination=output_path, format="turtle")
    print(f"Knowledge Graph saved to {output_path}")
    return g

create_rdf_graph("nel_results.csv")

Knowledge Graph saved to knowledge_graph.ttl


<Graph identifier=Ndfeb74cad4344fd7ab90879067b04296 (<class 'rdflib.graph.Graph'>)>

In [ ]:
from pyvis.network import Network
from rdflib import Graph, URIRef, Literal
from urllib.parse import urlparse, unquote

g = Graph()
g.parse("knowledge_graph.ttl", format="turtle")

net = Network(notebook=True, height="750px", width="100%", directed=True)

predicate_labels = {
    str(RDF.type): "is a",
    str(SCHEMA.mentions): "mentions",
    str(SCHEMA.name): "name",
    str(NST.normalizedForm): "normalized form"
}

labels_cache = {}

def get_clean_label(uri_or_literal):
    if isinstance(uri_or_literal, Literal):
        return str(uri_or_literal)

    if uri_or_literal in labels_cache:
        return labels_cache[uri_or_literal]

    parsed = urlparse(str(uri_or_literal))
    fragments = parsed.path.split('/')
    label = fragments[-1].split('#')[-1]

    label = unquote(label)

    label = label.replace('_', ' ').replace('+', ' ')

    labels_cache[uri_or_literal] = label
    return label

for s, p, o in g:
    if isinstance(s, URIRef):
        s_label = get_clean_label(s)
        if "wikidata.org" in str(s):
            node_type = "ellipse"
            title = f"Wikidata Entity: {s}"
        else:
            node_type = "box"
            title = f"Document: {s}"
        net.add_node(str(s), label=s_label, title=title, shape=node_type)

    if isinstance(o, URIRef):
        o_label = get_clean_label(o)
        net.add_node(str(o), label=o_label, title=str(o), shape="ellipse")

for s, p, o in g:
    if isinstance(s, URIRef) and isinstance(o, URIRef):
        edge_label = predicate_labels.get(str(p), get_clean_label(p))
        net.add_edge(str(s), str(o), label=edge_label, title=str(p))

net.toggle_hide_edges_on_drag(True)
net.set_options("""
{
  "physics": {
    "stabilization": {
      "iterations": 100
    },
    "timestep": 0.5
  },
  "nodes": {
    "font": {
      "size": 14
    }
  },
  "edges": {
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    },
    "smooth": false
  }
}
""")

net.show("knowledge_graph.html")

knowledge_graph.html


In [ ]:
df_nel

,original_text,entity_text,entity_type,entity_normal_form,wikidata_qid,start,end
0,Взрыв газа произошел в одной из квартир пятиэт...,Луначарского,LOC,луначарский,Q747751,60,72
1,Взрыв газа произошел в одной из квартир пятиэт...,Петрозаводске,LOC,петрозаводск,Q1895,75,88
2,Взрыв газа произошел в одной из квартир пятиэт...,РИА Новости,ORG,РИА Новости,Q821172,152,163
3,Взрыв газа произошел в одной из квартир пятиэт...,МЧС,ORG,МЧС,Q2624171,194,197
4,Взрыв газа произошел в одной из квартир пятиэт...,РЕН ТВ,ORG,РЕН ТВ,Q1479649,446,452
5,Взрыв газа произошел в одной из квартир пятиэт...,ТАСС,ORG,ТАСС,Q223799,577,581
6,Взрыв газа произошел в одной из квартир пятиэт...,Пятигорске,LOC,пятигорск,Q41970,669,679
7,Взрыв газа произошел в одной из квартир пятиэт...,МЧС,ORG,МЧС,Q2624171,935,938
8,Взрыв газа произошел в одной из квартир пятиэт...,РБК,ORG,РБК,Q629733,1004,1007
9,Взрыв газа произошел в одной из квартир пятиэт...,Telegram,ORG,Telegram,Q15616276,1010,1018
